# Example: Parsing OSM Data Directly with Databricks

> Use some Spatial-Utils functions to processing probable routes between NYC Citibike stations in SEP 2024.

__Notes__

* This example was run using DBR 16.0 on a 3 worker cluster (each with 8 CPUs and 61GB RAM);
* Uses 'spatial-utils-v1' branch with extra packages [route,viz].
* Requires a cluster with product `ST_` spatial sql functions enabled for KeplerGL Viz (may require photon cluster + spatial sql flag enabled).
* For the notebook to render well in github, we add screenshots of the map rendering and charts as well as artificially limit tabular results; when you run the notebook in databricks, you can uncomment and remove limits if desired.
* Requires you to download the [OSM PBF file for Andorra](https://download.geofabrik.de/europe/andorra.html) and upload this to a Volume. 

---
__Author:__ Timo Roest <timo@databricks.com> | _Last Modified:_ 14 MAR 2025

## Setup

In [0]:
%pip install -U "databricks-spatial[route,viz] @ git+https://github.com/databrickslabs/mosaic.git@spatial-utils-v1" --quiet

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

_Contact us to get spatial sql enabled, if needed; it is in private preview as of this example._ 

In [0]:
%run ./spatial_sql_flag

In [0]:
from spatial.route import pbf_source
from pyspark.sql import functions as F

from spatial.viz.keplergl import *
from spatial.viz.helpers import *

kviz = KeplerViz(spark)

In [0]:
# - CATALOG & SCHEMA
catalog = "geospatial_docs"
schema = "pbf_source"

# - TABLES
geometryType = "WKT"
bronzeTable = f"{catalog}.{schema}.andorra_bronze"
silverTable = f"{catalog}.{schema}.andorra_silver"
goldTable = f"{catalog}.{schema}.andorra_gold"

# - PATHS
pbfPath = f"/Volumes/{catalog}/{schema}/osm_files/andorra-latest.osm.pbf"

# - VARIABLES
h3Resolution = 8

## Ingest OSM Data

In [0]:
(
    spark.read.format("pbf")
    .option("path", pbfPath)
    .option("geometryType", "WKT")
    .load()
    .write.format("delta")
    .mode("overwrite")
    .saveAsTable(bronzeTable)
)

In [0]:
df_bronze = spark.read.table(bronzeTable)
display(df_bronze)

id,type,geometry,tags
625050,n,POINT(1.5721059 42.5299751),Map(highway -> crossing)
625054,n,POINT(1.5754728 42.5325841),Map(highway -> crossing)
625056,n,POINT(1.5784214 42.5330044),"Map(bus -> yes, highway -> bus_stop, public_transport -> stop_position)"
625063,n,POINT(1.5847204 42.5368336),"Map(crossing -> uncontrolled, highway -> crossing)"
625074,n,POINT(1.588504 42.5355669),"Map(crossing -> uncontrolled, highway -> crossing)"
625257,n,POINT(1.5977444 42.5658949),"Map(crossing -> zebra, crossing:island -> no, highway -> crossing)"
625268,n,POINT(1.6070441 42.5708747),"Map(crossing -> uncontrolled, crossing:markings -> no, highway -> crossing)"
26864258,n,POINT(1.3978986 42.666952),"Map(name -> Pica d'Estats, source -> Institut Cartogràfic de Catalunya, url:source -> ftp://geofons.icc.cat/fitxes/100CIMS/271064006.pdf, wikidata -> Q1537733, wikipedia:ca -> Pica d'Estats, ele -> 3143.45, wikipedia -> es:Pica d'Estats, prominence -> 1264, wikipedia:fr -> Pique d'Estats, source:prominence -> 3143-1879 (Pla de Beret), source:date -> 2010, name:es -> Pica d'Estats, natural -> peak, name:ca -> Pica d'Estats, summit:cross -> yes, name:fr -> Pique d'Estats)"
45160922,n,POINT(1.5187225 42.6454747),"Map(ele -> 2682, name -> Pic des Fangasses, natural -> peak, wikidata -> Q115677991)"
45160950,n,POINT(1.4341742 42.590256),"Map(name -> Agulla de Baiau, source -> Institut Cartogràfic i Geològic de Catalunya, wikidata -> Q21330590, ele -> 2860.5, wikipedia -> ca:Agulla de Baiau, natural -> peak, name:ca -> Agulla de Baiau)"


In [0]:
displayHTML(kviz.map_render(df_bronze.limit(1_000), "geometry"))

<!doctype html> Kepler.gl

## Classify Buildings into Commercial and Residential

In [0]:
# Explode the tags column to create key-value pairs
df_exploded_tags = (
    df_bronze
    .withColumn("key", F.explode(F.map_keys("tags")))
    .withColumn("value", F.col("tags")[F.col("key")])
)

# Define filters for residential and commercial buildings
residential_values = ["house", "residential", "detached", "terrace", "semidetached_house"]
commercial_values = ["commercial", "office", "retail", "service"]

# Explode the tags column and categorize buildings
df_categorized_buildings = (
    df_bronze.withColumn("key", F.explode(F.map_keys("tags")))
    .withColumn("value", F.col("tags")[F.col("key")])
    .filter((F.col("key") == "building") & F.col("geometry").isNotNull())
    .withColumn(
        "category",
        F.when(F.col("value").isin(*residential_values), "Residential")
         .when(F.col("value").isin(*commercial_values), "Commercial")
         .otherwise(None)
    )
    .filter(F.col("category").isNotNull())
)

# Visualise Sample
display(df_categorized_buildings.limit(10))

id,type,geometry,tags,key,value,category
3265137527,n,POINT(1.5257713 42.5051866),"Map(addr:street -> Avinguda de Tarragona, addr:postcode -> AD500, amenity -> courthouse, building -> house, addr:city -> Andorra la Vella)",building,house,Residential
389112078,a,"MULTIPOLYGON(((1.4811236 42.4475451,1.4811237 42.44743,1.4811561 42.4473252,1.481196 42.4472432,1.4814267 42.4469107,1.4815447 42.4468236,1.4817337 42.4468125,1.4818813 42.4468046,1.481868 42.4479996,1.4817937 42.4480001,1.4815876 42.448003,1.4812175 42.4477776,1.4811236 42.4475451)))","Map(disused:building -> retail, building:2020 -> retail, name:2020 -> E. Leclerc - Punt de Trobada, building -> retail, operator:2020 -> E. Leclerc, shop:2020 -> supermarket)",building,retail,Commercial
450295098,a,"MULTIPOLYGON(((1.4908238 42.5371997,1.490987 42.5369842,1.4915288 42.5371963,1.4915152 42.5372197,1.4913407 42.5374202,1.4908238 42.5371997)))",Map(building -> service),building,service,Commercial
450295100,a,"MULTIPOLYGON(((1.4912568 42.53591,1.4916445 42.5358983,1.4916649 42.5360938,1.4919324 42.536219,1.4918621 42.536356,1.4916195 42.5362792,1.4912727 42.5362825,1.4912568 42.53591)))","Map(building -> retail, name -> La Caubella)",building,retail,Commercial
450586548,a,"MULTIPOLYGON(((1.4908404 42.5366265,1.4908821 42.5364871,1.4909719 42.5363985,1.4911162 42.5364422,1.4910056 42.5366619,1.4908404 42.5366265)))",Map(building -> retail),building,retail,Commercial
495984564,a,"MULTIPOLYGON(((1.5317219 42.5091737,1.5318077 42.5086643,1.5318618 42.5083429,1.5323442 42.5083861,1.5322635 42.5088699,1.5321874 42.5093336,1.5320724 42.5093242,1.5320747 42.5093071,1.5318625 42.5092419,1.5318695 42.5091866,1.5317219 42.5091737)))","Map(building -> retail, name -> Centre Comercial Leclerc, operator -> E. Leclerc, shop -> mall)",building,retail,Commercial
495989050,a,"MULTIPOLYGON(((1.5397179 42.5111841,1.5398937 42.5111428,1.5399775 42.5113349,1.5398004 42.5113762,1.5397179 42.5111841)))",Map(building -> residential),building,residential,Residential
496004432,a,"MULTIPOLYGON(((1.5382407 42.5109478,1.5383363 42.5109101,1.5384269 42.5108744,1.5385685 42.5110698,1.5383824 42.5111431,1.5382407 42.5109478)))",Map(building -> residential),building,residential,Residential
568557738,a,"MULTIPOLYGON(((1.5274584 42.5080982,1.5274683 42.5078897,1.5276911 42.5077816,1.5278728 42.5079892,1.5278809 42.5079998,1.5278865 42.5080114,1.5278894 42.5080234,1.5278895 42.5080356,1.5278869 42.5080477,1.5278816 42.5080593,1.527872 42.508072,1.5278592 42.5080831,1.5278436 42.5080921,1.5278259 42.5080986,1.5278069 42.5081024,1.5277871 42.5081034,1.5274584 42.5080982)))","Map(building -> office, name -> Cambra de Comerç, Indústria i Serveis d'Andorra, name:ca -> Cambra de Comerç, Indústria i Serveis d'Andorra)",building,office,Commercial
586196768,a,"MULTIPOLYGON(((1.5111261 42.5505006,1.5112607 42.5504062,1.5114726 42.5505374,1.5113594 42.5506396,1.5113166 42.5506318,1.5111261 42.5505006)))",Map(building -> residential),building,residential,Residential


## Add H3 index
This allows us to easily compare and aggregate the data at a fine spatial resolution.

In [0]:
# Add H3 indices to all categorized buildings
df_categorized_buildings_indexed = df_categorized_buildings.withColumn(
    "h3_indices",
    F.expr(f"""
        CASE
            WHEN ST_GeometryType(geometry) = 'ST_Point'
                THEN ARRAY(h3_pointash3string(geometry, {h3Resolution}))
            WHEN ST_GeometryType(geometry) = 'ST_Polygon'
                THEN h3_coverash3string(geometry, {h3Resolution})
            WHEN ST_GeometryType(geometry) = 'ST_MultiPolygon'
                THEN h3_coverash3string(geometry, {h3Resolution})
            ELSE ARRAY()
        END
    """)
)


# Flatten H3 indices for aggregation
df_h3_flattened = df_categorized_buildings_indexed.withColumn("h3_index", F.explode("h3_indices"))

# Visualise Sample
display(df_h3_flattened.limit(10))

id,type,geometry,tags,key,value,category,h3_indices,h3_index
3265137527,n,POINT(1.5257713 42.5051866),"Map(addr:street -> Avinguda de Tarragona, addr:postcode -> AD500, amenity -> courthouse, building -> house, addr:city -> Andorra la Vella)",building,house,Residential,List(8839622325fffff),8839622325fffff
389112078,a,"MULTIPOLYGON(((1.4811236 42.4475451,1.4811237 42.44743,1.4811561 42.4473252,1.481196 42.4472432,1.4814267 42.4469107,1.4815447 42.4468236,1.4817337 42.4468125,1.4818813 42.4468046,1.481868 42.4479996,1.4817937 42.4480001,1.4815876 42.448003,1.4812175 42.4477776,1.4811236 42.4475451)))","Map(disused:building -> retail, building:2020 -> retail, name:2020 -> E. Leclerc - Punt de Trobada, building -> retail, operator:2020 -> E. Leclerc, shop:2020 -> supermarket)",building,retail,Commercial,List(883962399bfffff),883962399bfffff
450295098,a,"MULTIPOLYGON(((1.4908238 42.5371997,1.490987 42.5369842,1.4915288 42.5371963,1.4915152 42.5372197,1.4913407 42.5374202,1.4908238 42.5371997)))",Map(building -> service),building,service,Commercial,List(8839623a95fffff),8839623a95fffff
450295100,a,"MULTIPOLYGON(((1.4912568 42.53591,1.4916445 42.5358983,1.4916649 42.5360938,1.4919324 42.536219,1.4918621 42.536356,1.4916195 42.5362792,1.4912727 42.5362825,1.4912568 42.53591)))","Map(building -> retail, name -> La Caubella)",building,retail,Commercial,List(8839623a95fffff),8839623a95fffff
450586548,a,"MULTIPOLYGON(((1.4908404 42.5366265,1.4908821 42.5364871,1.4909719 42.5363985,1.4911162 42.5364422,1.4910056 42.5366619,1.4908404 42.5366265)))",Map(building -> retail),building,retail,Commercial,List(8839623a95fffff),8839623a95fffff
495984564,a,"MULTIPOLYGON(((1.5317219 42.5091737,1.5318077 42.5086643,1.5318618 42.5083429,1.5323442 42.5083861,1.5322635 42.5088699,1.5321874 42.5093336,1.5320724 42.5093242,1.5320747 42.5093071,1.5318625 42.5092419,1.5318695 42.5091866,1.5317219 42.5091737)))","Map(building -> retail, name -> Centre Comercial Leclerc, operator -> E. Leclerc, shop -> mall)",building,retail,Commercial,List(8839622321fffff),8839622321fffff
495989050,a,"MULTIPOLYGON(((1.5397179 42.5111841,1.5398937 42.5111428,1.5399775 42.5113349,1.5398004 42.5113762,1.5397179 42.5111841)))",Map(building -> residential),building,residential,Residential,List(8839622323fffff),8839622323fffff
496004432,a,"MULTIPOLYGON(((1.5382407 42.5109478,1.5383363 42.5109101,1.5384269 42.5108744,1.5385685 42.5110698,1.5383824 42.5111431,1.5382407 42.5109478)))",Map(building -> residential),building,residential,Residential,List(8839622323fffff),8839622323fffff
568557738,a,"MULTIPOLYGON(((1.5274584 42.5080982,1.5274683 42.5078897,1.5276911 42.5077816,1.5278728 42.5079892,1.5278809 42.5079998,1.5278865 42.5080114,1.5278894 42.5080234,1.5278895 42.5080356,1.5278869 42.5080477,1.5278816 42.5080593,1.527872 42.508072,1.5278592 42.5080831,1.5278436 42.5080921,1.5278259 42.5080986,1.5278069 42.5081024,1.5277871 42.5081034,1.5274584 42.5080982)))","Map(building -> office, name -> Cambra de Comerç, Indústria i Serveis d'Andorra, name:ca -> Cambra de Comerç, Indústria i Serveis d'Andorra)",building,office,Commercial,List(8839622321fffff),8839622321fffff
586196768,a,"MULTIPOLYGON(((1.5111261 42.5505006,1.5112607 42.5504062,1.5114726 42.5505374,1.5113594 42.5506396,1.5113166 42.5506318,1.5111261 42.5505006)))",Map(building -> residential),building,residential,Residential,List(883962234dfffff),883962234dfffff


## Further Analysis

This section calculates a density for residential and commercial buildings per H3 index, and classifies the index as being dominant of one or the other. 

In [0]:
# Calculate density (count) for residential and commercial buildings per H3 index
df_density = (
    df_h3_flattened
    .groupBy("h3_index", "category").count()
    .groupBy("h3_index")
    .pivot("category", ["Residential", "Commercial"])
    .sum("count")
    .fillna(0)
    .withColumnRenamed("Residential", "residential_count")
    .withColumnRenamed("Commercial", "commercial_count")
)

In [0]:
# Add dominance classification
df_density = df_density.withColumn(
    "dominance",
    F.when(F.col("residential_count") > F.col("commercial_count") * 2, "Residential Dominant")
     .when(F.col("commercial_count") > F.col("residential_count") * 2, "Commercial Dominant")
     .otherwise("Balanced")
)

In [0]:
# Identify densest areas
densest_residential = df_density.orderBy(F.desc("residential_count")).limit(10)
densest_commercial = df_density.orderBy(F.desc("commercial_count")).limit(10)

# Prepare dataset for visualization
df_output = (
  densest_residential
  .union(densest_commercial)
  .withColumn("wkt", F.expr("h3_boundaryaswkt(h3_index)"))
  .drop("h3_index")
)

In [0]:
displayHTML(
    kviz.map_render(df_output, "wkt")
)

<!doctype html> Kepler.gl